In [1]:
import _ct_district_grain_data_prep as _ct_district_data_prep

import geopandas as gpd
import pandas as pd
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS
from shared_utils import webmap_utils
from calitp_data_analysis import geography_utils
import branca
import branca.colormap as cm

In [2]:
import google.auth
credentials, project = google.auth.default()
import gcsfs
fs = gcsfs.GCSFileSystem()

## Operator

In [3]:
analysis_name = "Alameda-Contra Costa Transit District"

In [4]:
OPERATOR_ROUTE_MAP = GTFS_DATA_DICT.digest_tables.operator_routes_map_report

In [5]:
route_df = gpd.read_parquet(
    f"{RT_SCHED_GCS}{OPERATOR_ROUTE_MAP}.parquet",
    filters=[[("analysis_name", "==", analysis_name)]],
    storage_options={"token": credentials.token},
)

In [6]:
route_df = route_df.reset_index(drop=False)
route_df = route_df.rename(columns={"index": "Number"})

In [7]:
len(route_df)

86

In [8]:
route_df.geometry = route_df.geometry.buffer(100)

In [9]:
analysis_name_edited = analysis_name.replace(" ","_").lower()

In [10]:
operator_file = f"{analysis_name_edited}_routes"

In [11]:
operator_file

'alameda-contra_costa_transit_district_routes'

In [12]:
color_map = cm.linear.Spectral_11.scale(0, 150)

In [13]:
transit_shn_map = webmap_utils.set_state_export(
    route_df[["Route","geometry", "Number"]],
    subfolder = "operator_digest/",
    filename=operator_file,
    map_title=f"Transit Routes for {analysis_name}",
    cmap=color_map,
    color_col="Number",
    
)

/home/jovyan/data-analyses/_shared_utils/shared_utils/webmap_utils.py:118: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())


In [14]:
webmap_utils.render_spa_link(transit_shn_map["spa_link"], text=f"Routes for {analysis_name}")

<a href="https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJUcmFuc2l0IFJvdXRlcyBmb3IgQWxhbWVkYS1Db250cmEgQ29zdGEgVHJhbnNpdCBEaXN0cmljdCIsICJ1cmwiOiAiaHR0cHM6Ly9zdG9yYWdlLmdvb2dsZWFwaXMuY29tL2NhbGl0cC1tYXAtdGlsZXMvb3BlcmF0b3JfZGlnZXN0L2FsYW1lZGEtY29udHJhX2Nvc3RhX3RyYW5zaXRfZGlzdHJpY3Rfcm91dGVzLmdlb2pzb24uZ3oiLCAicHJvcGVydGllcyI6IHsic3Ryb2tlZCI6IGZhbHNlLCAiaGlnaGxpZ2h0X3NhdHVyYXRpb25fbXVsdGlwbGllciI6IDAuNX19XSwgImxhdF9sb24iOiBbMzcuNzYxNjIyNzc3OTQwOTksIC0xMjIuMjAzNjAxMTE1MTk2MjFdLCAiem9vbSI6IDEzfQ==" target="_blank">Open Routes for Alameda-Contra Costa Transit District in New Tab</a>

In [15]:
webmap_utils.display_spa_map(transit_shn_map["spa_link"])

## CT District

In [ ]:
district = "04 - Bay Area / Oakland"

In [ ]:
district_int = [int(s) for s in district.split() if s.isdigit()][0]

In [ ]:
boundary_file = f"district_{district_int}_boundary"
transit_routes_file = f"district_{district_int}_transit_routes"
shn_file = f"district_{district_int}_shn"
transit_shn_file = f"district_{district_int}_transit_routes_shn"

In [ ]:
operator_df = _ct_district_data_prep.data_wrangling_operator_profile(district)

In [ ]:
district_gdf = _ct_district_data_prep.load_ct_district(district_int)

In [ ]:
transit_route_shs_gdf, transit_route_shs_table = (
    _ct_district_data_prep.final_transit_route_shs_outputs(15, district_int)
)

In [ ]:
transit_route_shs_gdf.columns

In [ ]:
operator_gdf = _ct_district_data_prep.data_wrangling_operator_map(
    list(operator_df.analysis_name.unique())
)

In [ ]:
shn_gdf = _ct_district_data_prep.load_buffered_shn_map(district_int)

In [ ]:
color_map2 = cm.linear.RdYlBu_11.scale(0, 50)

In [ ]:
shn_map = webmap_utils.set_state_export(
    shn_gdf,
    subfolder = "caltrans_district_digest/",
    filename=shn_file,
    map_title="State Highway Network Map",
    map_type='state_highway_network'
)

In [ ]:
transit_shn_map = webmap_utils.set_state_export(
    transit_route_shs_gdf,
    subfolder = "caltrans_district_digest/",
    filename=transit_shn_file,
    map_title="Transit Routes on the State Highway Network",
    cmap=color_map2,
    color_col="Number",
    existing_state=shn_map
    
)

In [ ]:
webmap_utils.render_spa_link(transit_shn_map["spa_link"], text="Open Routes on State Highway System Map")